## Using Plotly Express to create a globally COVID-19 confirmed cases bubble map that is playable and interactive.

In [29]:
import pandas as pd
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

### About Plotly Express
#### Plotly: The front-end for ML and data science models
#### The leading front-end for ML & data science models in Python, R, and Julia. As the leading UI layer for ML and data…

If you have a lot of customised requirements, Plotly Express is not your choice. You may need to check out the regular Plotly library APIs if so. Otherwise, Plotly Express is highly recommended because it saves you time and produces as fabulous as the regular APIs do.
#### COVID-19 Data Source

Many organisations are providing up-to-date COVID-19 data at the moment. I found this one from the European Union:
Download today’s data on the geographic distribution of COVID-19 cases worldwide
Every day between 6.00 and 10.00 CET, a team of epidemiologists screens up to 500 relevant sources to collect the…
www.ecdc.europa.eu
#### Disclamation: 
The data sources are simply found via search engines, this article uses the COVID-19 data for the data visualisation techniques demonstration only, so I’m NOT responsible for the accuracy of the data.
You may download the CSV for the above link. However, we can directly read an URL in Python using Pandas.

In [30]:
df.sample(10)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continentExp
10375,20/04/2020,20,4,2020,6060,48,Russia,RU,RUS,144478050.0,Europe
12266,10/02/2020,10,2,2020,0,0,Thailand,TH,THA,69428524.0,Asia
3704,23/02/2020,23,2,2020,0,0,Ecuador,EC,ECU,17084357.0,America
9666,07/04/2020,7,4,2020,36,0,Palestine,PS,PSE,4569087.0,Asia
8551,30/03/2020,30,3,2020,0,0,Myanmar,MM,MMR,53708395.0,Asia
8227,10/04/2020,10,4,2020,3,0,Monaco,MC,MCO,38682.0,Europe
3738,20/01/2020,20,1,2020,0,0,Ecuador,EC,ECU,17084357.0,America
10064,12/04/2020,12,4,2020,515,35,Portugal,PT,PRT,10281762.0,Europe
12698,01/03/2020,1,3,2020,2,0,United_Arab_Emirates,AE,ARE,9630959.0,Asia
3409,18/02/2020,18,2,2020,0,0,Denmark,DK,DNK,5797446.0,Europe


### In this tutorial, we are going to use the following fields in the dataset:
1. date
2. number of cases
3. number of deaths
4. country
5. continent

Therefore, let’s just remove the unuseful columns and rename the columns as preferences. Please note that this step is optional, but it is a good manner to keep your Pandas data frame tidy.

In [31]:
# Remove unuseful columns
df = df[['dateRep', 'cases', 'deaths', 'countriesAndTerritories', 'countryterritoryCode', 'continentExp']]
# Rename columns
df = df.rename(columns={
    'dateRep': 'date',
    'countriesAndTerritories': 'country',
    'countryterritoryCode': 'countryCode',
    'continentExp': 'continent'
})
# Convert string to datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
# Preview the data frame
df.sample(10)

,date,cases,deaths,country,countryCode,continent
7708,2020-04-12,3,1,Malawi,MWI,Africa
11534,2020-03-16,2000,152,Spain,ESP,Europe
9627,2020-01-18,0,0,Pakistan,PAK,Asia
750,2020-04-20,48,9,Austria,AUT,Europe
10555,2020-04-03,0,0,Saint_Kitts_and_Nevis,KNA,America
5671,2020-02-11,0,0,Iceland,ISL,Europe
3679,2020-03-24,192,4,Ecuador,ECU,America
3154,2020-04-01,32,1,Cyprus,CYP,Europe
10347,2020-01-20,0,0,Romania,ROU,Europe
2565,2020-03-27,2,0,Chad,TCD,Africa



### Create the Bubble Map

Before creating the interactive map, let’s create a static map first. Later on, you will be surprised how easy it is to create a playable interactive map.
For the static map, I would like to create it for “today”. So, let’s get the data frame for all the countries globally, but only for “today”.


In [32]:
from datetime import datetime
# Get today as string
today = datetime.now().strftime('%Y-%m-%d')
# Get a data frame only for today
df_today = df[df.date == '2020-04-27']
# Preview the data frame
df_today.head()

,date,cases,deaths,country,countryCode,continent
0,2020-04-27,68,21,Afghanistan,AFG,Asia
109,2020-04-27,14,1,Albania,ALB,Europe
159,2020-04-27,126,6,Algeria,DZA,Africa
273,2020-04-27,7,0,Andorra,AND,Europe
318,2020-04-27,1,0,Angola,AGO,Africa


In [34]:
# My current day is 27 April, so the data frame I got is shown as above.
# Then, let’s plot the data frame using Plotly Express.
today='2020-04-27'
import plotly.express as px
fig = px.scatter_geo(
    df_today, 
    locations='countryCode',
    color='continent',
    hover_name='country',
    size='cases',
    projection="natural earth",
    title=f'World COVID-19 Cases for {today}'
)
fig.show()

##### Simply the above 3 lines (Well, there are 12 lines here because I want to show them with better readability), the global map with bubbles that indicate the number of confirmed cases for today is shown as follows.


## Let me explain the parameters of the px.scatter_geo() function.
1. For the first parameter, we need to provide the Pandas data frame. Plotly Express is fully compatible with Pandas so that we could always use data frames.

2. locations takes the country codes to geocode the countries on the map. Please note that we simply let Plotly Express know that the column named “countryCode” is the column indicating locations. We can also use latitude and longitude. However, since the data source has provided the “ISO-Alpha” country codes so that it is much easier to use it.

3. color is optional here, but it is good to show the bubbles with different colour codes depending on different continents, which makes the graph clearer.

4. hover_name decided what to display when the mouse hovering on the bubble. Here we let it display the country name, so we specify the column name “country” from the data frame.

5. size is an important parameter to determine how large the bubble is on the map. Of course, we want it to indicate how many cases in each country, so we specify “cases”.

6. projection tells Plotly Express the presentation of the map we want to use. There are many other options such as “equirectangular” which will show the map in a rectangle as follows.


In [35]:
# Convert date to string type
df['date'] = pd.to_datetime(df["date"])

df['date'] = df.date.dt.strftime('%Y-%m-%d')
# Sort the data frame on date
df = df.sort_values(by=['date'])
# Some countries does not have code, let's drop all the invalid rows
df = df.dropna()
# Preview the data frame
df.head(10)

,date,cases,deaths,country,countryCode,continent
4108,2019-12-31,0,0,Estonia,EST,Europe
13039,2019-12-31,0,0,United_States_of_America,USA,America
9531,2019-12-31,0,0,Oman,OMN,Asia
2349,2019-12-31,0,0,Canada,CAN,America
10367,2019-12-31,0,0,Romania,ROU,Europe
12195,2019-12-31,0,0,Taiwan,TWN,Asia
1872,2019-12-31,0,0,Brazil,BRA,America
9645,2019-12-31,0,0,Pakistan,PAK,Asia
9418,2019-12-31,0,0,Norway,NOR,Europe
2745,2019-12-31,27,0,China,CHN,Asia


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13270 entries, 4108 to 0
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         13270 non-null  object
 1   cases        13270 non-null  int64 
 2   deaths       13270 non-null  int64 
 3   country      13270 non-null  object
 4   countryCode  13270 non-null  object
 5   continent    13270 non-null  object
dtypes: int64(2), object(4)
memory usage: 725.7+ KB



#### OK. The data frame is ready now. Please refer to the previous px.scatter_geo() function that we created for today’s data only. To make it “playable”, simply let Plotly knows which column will be utilised for “playing” animation_frame=”date”. The full function is as follows.


In [41]:
fig = px.scatter_geo(
    df_today, 
    locations='countryCode',
    color='continent',
    hover_name='country',
    size='cases',
    projection="natural earth",
    title=f'World COVID-19 Cases',
    animation_frame="date"
)
fig.show()


### We can also drag the progress bar to quickly navigate to certain dates, as well as interact to the map to zoom in/out and panning.

### See how easy it is! Thanks to Plotly, we can create fancy graphs in seconds.

### Summary

#### Indeed, the features of Plotly Express is kind of limited. For example, if we want to customise the scale of bubble size on the map (some countries with fewer cases are not visible), it will not be convenient.
However, the limitations are trade-off because Plotly Express meant to provide terse API to create beautiful graphs very quickly. 

#### So, we can choose to use Plotly regular APIs such as the Graph Objects to achieve those customised requirements.

